In [1]:
import os
import json
import pandas as pd

In [2]:
folder_path = './우울증_1-10'

In [3]:
output_folder = './우울증전처리'

공통 정보 컬럼
-'paragraph_speaker': 내담자/상담사 여부
-'paragraph_text': 상담 문장 내용

우울 특화 컬럼
-'depressive_mood': 우울감
-'worthlessness': 무가치감
-'guilt': 죄책감
-'impaired_cognition': 인지 장애
-'suicidal': 자살사고
-'anhedonia': 무쾌감
-'psychomotor_changes': 정신운동 변화
-'weight_appetite': 체중·식욕 변화
-'sleep_disturbance': 수면 장애
-'fatigue': 피로감

불안 특화 컬럼
-'anxiety_mood': 불안감
-'derealization': 이인증(비현실감)
-'perceived_loss_of_control': 통제력 상실 인식
-'anxiety_control': 불안 통제
-'concentration': 집중력
-'avoidance': 회피 경향
-'physical_symptoms': 신체 증상
-'irritability': 신경 예민

중독 특화 
'loss_of_control': 통제력 상실
'craving': 갈망
'lying': 거짓말 행동
'tolerance': 내성
'withdrawal': 금단 증상
'salience': 중독 중요성(삶에서 차지하는 정도)
'resource_investment': 자원 투자(중독에 투입한 시간/비용 등)
'daily_functioning': 일상 기능 저해
'social_problems': 사회적 문제 발생
'negative_consequences': 부정적 결과

긍정적인 지표 컬럼 없음 

In [4]:
import re

cols = ['paragraph_speaker','paragraph_text','depressive_mood','worthlessness','guilt','impaired_cognition','suicidal','anhedonia','psychomotor_changes','weight_appetite','sleep_disturbance','fatigue']
symptom_cols = ['depressive_mood','worthlessness','guilt','impaired_cognition','suicidal','anhedonia','psychomotor_changes','weight_appetite','sleep_disturbance','fatigue']

def is_survey_or_rating(text):
    survey_keywords = ['네', '아니요', '모름', '없음', '예', '아니오', '있음', '잘 모르겠음']
    rating_pattern = r'^[0-9]+점\\.?$'
    text_clean = str(text).strip()
    if re.match(rating_pattern, text_clean):
        return True
    if text_clean in survey_keywords:
        return True
    return False

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            paragraphs = data['paragraph']
            df = pd.DataFrame(paragraphs)
            df_selected = df[cols]

            # 설문/평가 텍스트 행 제거
            df_selected = df_selected[~df_selected['paragraph_text'].apply(is_survey_or_rating)]

            # 지정 증상 컬럼 중 하나라도 0이 아닌 행만 필터링
            df_filtered = df_selected[(df_selected[symptom_cols] != 0).any(axis=1)]

            output_path = os.path.join(output_folder, filename.replace('.json', '.csv'))
            df_filtered.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f'{filename}: 완료, {len(df_filtered)}개 행')

        except Exception as e:
            print(f'{filename}: 에러 발생 - {str(e)}')
            continue

label_depression_10_check_D035.json: 완료, 11개 행
label_depression_10_check_D036.json: 완료, 6개 행
label_depression_10_check_D039.json: 완료, 12개 행
label_depression_10_check_D075.json: 완료, 1개 행
label_depression_1_check_D002.json: 완료, 14개 행
label_depression_1_check_D003.json: 완료, 12개 행
label_depression_1_check_D004.json: 완료, 24개 행
label_depression_1_check_D005.json: 완료, 12개 행
label_depression_1_check_D006.json: 완료, 10개 행
label_depression_1_check_D007.json: 완료, 23개 행
label_depression_1_check_D012.json: 완료, 46개 행
label_depression_1_check_D014.json: 완료, 34개 행
label_depression_1_check_D018.json: 완료, 7개 행
label_depression_1_check_D019.json: 완료, 27개 행
label_depression_1_check_D026.json: 완료, 13개 행
label_depression_1_check_D027.json: 완료, 14개 행
label_depression_1_check_D028.json: 완료, 13개 행
label_depression_1_check_D029.json: 완료, 8개 행
label_depression_1_check_D032.json: 완료, 13개 행
label_depression_1_check_D033.json: 완료, 33개 행
label_depression_1_check_D034.json: 완료, 15개 행
label_depression_1_check_D035.json

In [5]:
!pip install -q sentence_transformers

In [6]:
!pip install -q konlpy

In [7]:
import os
import re
import pandas as pd
from konlpy.tag import Okt
from sentence_transformers import SentenceTransformer

# 1. 텍스트 정규화 함수
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

# 2. 한국어 토큰화
okt = Okt()
def tokenize(text):
    return okt.morphs(text)

# 3. 벡터 임베딩 모델 로드
embedding_model = SentenceTransformer('sentence-transformers/LaBSE')

# 처리 대상 폴더
folder_path = '우울증전처리'  # CSV 파일들이 위치한 폴더 경로

# 결과 저장용 딕셔너리(파일명 -> 문장별 임베딩)
all_embeddings = {}

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)

        # 'paragraph_text' 컬럼 기준 처리
        normalized_texts = df['paragraph_text'].astype(str).apply(normalize_text)
        tokenized_texts = normalized_texts.apply(tokenize).apply(lambda tokens: ' '.join(tokens))

        # 임베딩 생성
        embeddings = embedding_model.encode(tokenized_texts.tolist())

        # 결과 저장
        all_embeddings[file] = embeddings

        print(f'{file} 처리 완료, 문장 수: {len(normalized_texts)}')

# all_embeddings는 각 파일별로 문장 임베딩 배열 저장


c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


label_depression_10_check_D035.csv 처리 완료, 문장 수: 11
label_depression_10_check_D036.csv 처리 완료, 문장 수: 6
label_depression_10_check_D039.csv 처리 완료, 문장 수: 12
label_depression_10_check_D075.csv 처리 완료, 문장 수: 1
label_depression_1_check_D002.csv 처리 완료, 문장 수: 14
label_depression_1_check_D003.csv 처리 완료, 문장 수: 12
label_depression_1_check_D004.csv 처리 완료, 문장 수: 24
label_depression_1_check_D005.csv 처리 완료, 문장 수: 12
label_depression_1_check_D006.csv 처리 완료, 문장 수: 10
label_depression_1_check_D007.csv 처리 완료, 문장 수: 23
label_depression_1_check_D012.csv 처리 완료, 문장 수: 46
label_depression_1_check_D014.csv 처리 완료, 문장 수: 34
label_depression_1_check_D018.csv 처리 완료, 문장 수: 7
label_depression_1_check_D019.csv 처리 완료, 문장 수: 27
label_depression_1_check_D026.csv 처리 완료, 문장 수: 13
label_depression_1_check_D027.csv 처리 완료, 문장 수: 14
label_depression_1_check_D028.csv 처리 완료, 문장 수: 13
label_depression_1_check_D029.csv 처리 완료, 문장 수: 8
label_depression_1_check_D032.csv 처리 완료, 문장 수: 13
label_depression_1_check_D033.csv 처리 완료, 문장 수: 33


In [8]:
import pickle

with open('depdata_all_embeddings.pkl', 'wb') as f:
    pickle.dump(all_embeddings, f)
    
with open('depdata_all_embeddings.pkl', 'rb') as f:
    all_embeddings = pickle.load(f)
